In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
fh_data = pd.read_csv('flowhub_location_analysis_0426.csv',index_col = 0)

In [4]:
fh_data = fh_data.sort_values(by = ['location_id','year_month'])

In [5]:
fh_data = fh_data.reset_index(drop = True)

In [6]:
fh_data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796
...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876


In [7]:
def all_time_mean(d):
    return d.reset_index(drop = True)['margin_final'].cumsum() / [i + 1 for i in list(range(len(d)))]

In [8]:
gm_9m = fh_data[['location_id','year_month','margin_final']].groupby('location_id').apply(lambda x: all_time_mean(x))

In [9]:
fh_data['gm_9m'] = gm_9m.values

In [10]:
fh_data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493,0.462855
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390,0.438033
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695,0.437449
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796,0.431918
...,...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991,0.578977
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042,0.572597
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060,0.567273
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876,0.565713


# GM %

In [11]:
gm_3m = fh_data[['location_id','year_month','margin_final']].groupby('location_id').apply(lambda x: x['margin_final'].rolling(3).mean()).to_frame()\
.reset_index().set_index('level_1')['margin_final']

In [12]:
gm_6m = fh_data[['location_id','year_month','margin_final']].groupby('location_id').apply(lambda x: x['margin_final'].rolling(6).mean()).to_frame()\
.reset_index().set_index('level_1')['margin_final']

In [13]:
fh_data = pd.concat([fh_data, gm_3m],axis=1)
fh_data = pd.concat([fh_data, gm_6m],axis=1)


In [14]:
fh_data.columns = ['year_month', 'subtotalInDollars', 'costInDollars',
       'subtotalInDollars_after_tax', 'profit', 'profit_after_tax',
       'margin_perc', 'margin_perc_after_tax', 'location_id', 'location_state',
       'margin_final','gm_9m','gm_3m','gm_6m']

In [15]:
fh_data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217,NaN,NaN
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493,0.462855,NaN,NaN
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390,0.438033,0.438033,NaN
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695,0.437449,0.394859,NaN
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796,0.431918,0.411294,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991,0.578977,0.518397,0.541009
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042,0.572597,0.497991,0.533655
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060,0.567273,0.483364,0.519047
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876,0.565713,0.512659,0.515528


In [16]:
gm_distr_3m = fh_data.groupby(['location_state','year_month'])['gm_3m'].describe().reset_index()
gm_distr_3m.index = pd.to_datetime(gm_distr_3m['year_month'])
gm_distr_3m

,location_state,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,,
2021-01-01,CA,2021-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-02-01,CA,2021-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-01,CA,2021-03,29.0,0.468210,0.178297,-0.400258,0.455200,0.484419,0.524206,0.640522
2021-04-01,CA,2021-04,30.0,0.467543,0.158449,-0.302268,0.450068,0.482467,0.521354,0.636632
2021-05-01,CA,2021-05,30.0,0.472111,0.143053,-0.210860,0.446379,0.484765,0.520414,0.630211
...,...,...,...,...,...,...,...,...,...,...
2021-12-01,OR,2021-12,85.0,0.473044,0.111719,-0.038694,0.437073,0.492875,0.550419,0.683116
2022-01-01,OR,2022-01,83.0,0.482975,0.094433,0.087171,0.441484,0.492362,0.548126,0.707622
2022-02-01,OR,2022-02,85.0,0.496086,0.090362,0.105356,0.455090,0.508144,0.560204,0.722869


In [17]:
round(gm_distr_3m[['25%','50%','75%','location_state']].groupby(['location_state']).mean(),2)

,25%,50%,75%
location_state,,,
CA,0.44,0.47,0.53
CO,0.37,0.46,0.53
MA,0.45,0.48,0.50
MI,0.32,0.42,0.48
OR,0.45,0.50,0.56


In [101]:
#round(fh_data['gm_3m'].quantile(0.95),2)

In [19]:
gm_distr_6m = fh_data.groupby(['location_state','year_month'])['gm_6m'].describe().reset_index()
gm_distr_6m.index = pd.to_datetime(gm_distr_6m['year_month'])
gm_distr_6m

,location_state,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,,
2021-01-01,CA,2021-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-02-01,CA,2021-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-01,CA,2021-03,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-01,CA,2021-04,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-01,CA,2021-05,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2021-12-01,OR,2021-12,82.0,0.481245,0.087848,0.068875,0.432842,0.490628,0.551793,0.676548
2022-01-01,OR,2022-01,80.0,0.486561,0.085812,0.057056,0.437636,0.496742,0.550733,0.685182
2022-02-01,OR,2022-02,80.0,0.488366,0.087762,0.056656,0.441366,0.498415,0.554336,0.689048


In [20]:
round(gm_distr_6m[['25%','50%','75%','location_state']].groupby(['location_state']).mean(),2)

,25%,50%,75%
location_state,,,
CA,0.43,0.47,0.52
CO,0.36,0.45,0.53
MA,0.45,0.48,0.49
MI,0.33,0.42,0.47
OR,0.44,0.50,0.55


In [102]:
#round(fh_data['gm_6m'].quantile(0.95),2)

In [22]:
gm_distr_9m = fh_data.groupby(['location_state','year_month'])['gm_9m'].describe().reset_index()
gm_distr_9m.index = pd.to_datetime(gm_distr_9m['year_month'])
gm_distr_9m

,location_state,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,,
2021-01-01,CA,2021-01,29.0,0.487757,0.105505,0.069108,0.445685,0.504216,0.529615,0.647808
2021-02-01,CA,2021-02,31.0,0.488158,0.094591,0.107034,0.457721,0.492693,0.526099,0.644929
2021-03-01,CA,2021-03,30.0,0.469392,0.175316,-0.400258,0.455501,0.487032,0.522088,0.640522
2021-04-01,CA,2021-04,30.0,0.472707,0.143189,-0.209424,0.452380,0.489776,0.522451,0.639426
2021-05-01,CA,2021-05,32.0,0.483494,0.120412,-0.083702,0.454868,0.494304,0.533439,0.636098
...,...,...,...,...,...,...,...,...,...,...
2021-12-01,OR,2021-12,90.0,0.486104,0.076877,0.159075,0.443492,0.494923,0.549015,0.673065
2022-01-01,OR,2022-01,88.0,0.491834,0.071579,0.158121,0.445635,0.502644,0.549949,0.680741
2022-02-01,OR,2022-02,89.0,0.493516,0.072318,0.158500,0.447989,0.502601,0.552101,0.682175


In [23]:
round(gm_distr_9m[['25%','50%','75%','location_state']].groupby(['location_state']).mean(),2)

,25%,50%,75%
location_state,,,
CA,0.44,0.48,0.52
CO,0.38,0.46,0.53
MA,0.45,0.48,0.50
MI,0.33,0.42,0.47
OR,0.45,0.50,0.55


In [103]:
#round(fh_data['gm_9m'].quantile(0.95),2)

# rev change

In [25]:
def gmv_change_variance_point_mapping(x):
    if x < -0.1:
        return -5
    elif x >= -0.1 and x <-0.05:
        return -2.5
    elif x >= -0.05 and x < 0:
        return 0
    elif x >= 0 and x < 0.05:
        return 5
    elif x >= 0.05 and x <= 0.5:
        return 10
    else:
        return numpy.nan
    

In [26]:
fh_data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217,NaN,NaN
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493,0.462855,NaN,NaN
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390,0.438033,0.438033,NaN
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695,0.437449,0.394859,NaN
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796,0.431918,0.411294,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991,0.578977,0.518397,0.541009
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042,0.572597,0.497991,0.533655
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060,0.567273,0.483364,0.519047
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876,0.565713,0.512659,0.515528


In [27]:
rev_change = fh_data[['location_id','year_month','subtotalInDollars']].groupby('location_id').apply(lambda x: x['subtotalInDollars'].pct_change()).to_frame()\
.reset_index().set_index('level_1')['subtotalInDollars']

In [28]:
fh_data = pd.concat([fh_data, rev_change],axis=1)
fh_data.columns = ['year_month', 'subtotalInDollars', 'costInDollars',
       'subtotalInDollars_after_tax', 'profit', 'profit_after_tax',
       'margin_perc', 'margin_perc_after_tax', 'location_id', 'location_state',
       'margin_final','gm_9m', 'gm_3m','gm_6m','rev_change']

In [29]:
fh_data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217,NaN,NaN,NaN
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493,0.462855,NaN,NaN,16.868939
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390,0.438033,0.438033,NaN,0.111717
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695,0.437449,0.394859,NaN,-0.183553
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796,0.431918,0.411294,NaN,-0.047323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991,0.578977,0.518397,0.541009,0.115825
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042,0.572597,0.497991,0.533655,-0.077624
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060,0.567273,0.483364,0.519047,0.049723
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876,0.565713,0.512659,0.515528,0.111215


In [30]:
rev_change_bm = fh_data.groupby(['location_state','year_month'])['rev_change'].describe().reset_index()
rev_change_bm

,location_state,year_month,count,mean,std,min,25%,50%,75%,max
0,CA,2021-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CA,2021-02,29.0,-0.031567,0.224824,-0.420327,-0.119670,-0.095220,-0.000571,0.917003
2,CA,2021-03,30.0,0.252242,0.181009,0.016118,0.157067,0.218236,0.281481,0.941027
3,CA,2021-04,30.0,0.019526,0.096235,-0.168459,-0.038297,-0.006050,0.086635,0.303247
4,CA,2021-05,31.0,0.026276,0.552399,-0.415112,-0.104708,-0.046000,-0.025454,2.956098
...,...,...,...,...,...,...,...,...,...,...
75,OR,2021-12,86.0,0.077924,0.248547,-0.930119,-0.010617,0.078079,0.145198,1.709458
76,OR,2022-01,87.0,0.010049,0.716716,-0.942823,-0.139552,-0.102357,-0.021386,5.076231
77,OR,2022-02,86.0,0.025033,0.513279,-0.797641,-0.064643,-0.009095,0.012136,4.617265
78,OR,2022-03,88.0,0.101515,0.280456,-0.984440,0.019888,0.067465,0.113755,1.770131


In [108]:
rev_change_bm = rev_change_bm[['year_month','50%','location_state']]
rev_change_bm_long = rev_change_bm[(rev_change_bm['year_month'] >= '2021-03') & (rev_change_bm['year_month'] <= '2022-03')]

rev_change_bm_weight = pd.DataFrame([0.04,0.12,0.2,0.28,0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1])
rev_change_bm_weight['year_month'] = rev_change_bm_long['year_month'].unique()
rev_change_bm_weight.columns = ['weight','year_month']

rev_change_bm_long_with_weight = pd.merge(rev_change_bm_long,rev_change_bm_weight,on = 'year_month')
rev_change_bm_long_with_weight[rev_change_bm_long_with_weight['location_state'] == 'CA']

,year_month,50%,location_state,weight
0,2021-03,0.218236,CA,0.04
5,2021-04,-0.006050,CA,0.12
10,2021-05,-0.046000,CA,0.20
15,2021-06,-0.110598,CA,0.28
20,2021-07,0.053840,CA,0.36
25,2021-08,-0.022247,CA,0.44
30,2021-09,-0.047323,CA,0.52
35,2021-10,0.011126,CA,0.60
40,2021-11,-0.081735,CA,0.68
45,2021-12,0.065987,CA,0.76


# sum 3m COGS

In [34]:
sum_3m_cogs = fh_data[['location_id','year_month','costInDollars']].groupby('location_id').apply(lambda x: x['costInDollars'].rolling(3).sum()).to_frame()\
.reset_index().set_index('level_1')['costInDollars']

In [35]:
fh_data = pd.concat([fh_data, sum_3m_cogs],axis=1)
fh_data.columns = ['year_month', 'subtotalInDollars', 'costInDollars',
       'subtotalInDollars_after_tax', 'profit', 'profit_after_tax',
       'margin_perc', 'margin_perc_after_tax', 'location_id', 'location_state',
       'margin_final', 'gm_9m','gm_3m','gm_6m','rev_change','sum_3m_cogs']

In [36]:
fh_data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217,NaN,NaN,NaN,NaN
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493,0.462855,NaN,NaN,16.868939,NaN
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390,0.438033,0.438033,NaN,0.111717,5467.50
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695,0.437449,0.394859,NaN,-0.183553,7452.50
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796,0.431918,0.411294,NaN,-0.047323,6927.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991,0.578977,0.518397,0.541009,0.115825,105191.95
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042,0.572597,0.497991,0.533655,-0.077624,109265.23
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060,0.567273,0.483364,0.519047,0.049723,115076.27
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876,0.565713,0.512659,0.515528,0.111215,110976.56


# inventory

In [37]:
inv_data = pd.read_csv('flowhub_location_inventory_0426.csv',index_col = 0)

In [38]:
inv_data.columns

Index(['loc_id', 'loc_state', 'inventory', 'num_sku_in_inventory', 'coverage',
       'non_zero_cost_perc'],
      dtype='object')

In [39]:
inv_data.columns = ['location_id','loc_state', 'inventory', 'num_sku_in_inventory', 'coverage',
       'non_zero_cost_perc']

# joined

In [40]:
data = pd.merge(fh_data,inv_data, on = ['location_id'],how = 'left')

In [41]:
data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217,NaN,NaN,NaN,NaN,CA,31891.67,272.0,1.0,1.0
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493,0.462855,NaN,NaN,16.868939,NaN,CA,31891.67,272.0,1.0,1.0
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390,0.438033,0.438033,NaN,0.111717,5467.50,CA,31891.67,272.0,1.0,1.0
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695,0.437449,0.394859,NaN,-0.183553,7452.50,CA,31891.67,272.0,1.0,1.0
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796,0.431918,0.411294,NaN,-0.047323,6927.00,CA,31891.67,272.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991,0.578977,0.518397,0.541009,0.115825,105191.95,CO,63879.09,481.0,1.0,1.0
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042,0.572597,0.497991,0.533655,-0.077624,109265.23,CO,63879.09,481.0,1.0,1.0
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060,0.567273,0.483364,0.519047,0.049723,115076.27,CO,63879.09,481.0,1.0,1.0
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876,0.565713,0.512659,0.515528,0.111215,110976.56,CO,63879.09,481.0,1.0,1.0


# inventory TO

In [42]:
data['inventory_to'] =  data['sum_3m_cogs'] / data['inventory'] * 4

In [43]:
data

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217,NaN,NaN,NaN,NaN,CA,31891.67,272.0,1.0,1.0,NaN
1,2021-06,3538.05,2602.00,4068.7575,936.05,1466.7575,0.264567,0.360493,22rkBxnqoaobkG9hr,CA,0.360493,0.462855,NaN,NaN,16.868939,NaN,CA,31891.67,272.0,1.0,1.0,NaN
2,2021-07,3933.31,2766.50,4523.3065,1166.81,1756.8065,0.296648,0.388390,22rkBxnqoaobkG9hr,CA,0.388390,0.438033,0.438033,NaN,0.111717,5467.50,CA,31891.67,272.0,1.0,1.0,0.685759
3,2021-08,3211.34,2084.00,3693.0410,1127.34,1609.0410,0.351050,0.435695,22rkBxnqoaobkG9hr,CA,0.435695,0.437449,0.394859,NaN,-0.183553,7452.50,CA,31891.67,272.0,1.0,1.0,0.934727
4,2021-09,3059.37,2076.50,3518.2755,982.87,1441.7755,0.321265,0.409796,22rkBxnqoaobkG9hr,CA,0.409796,0.431918,0.411294,NaN,-0.047323,6927.00,CA,31891.67,272.0,1.0,1.0,0.868816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-12,76982.03,41878.95,88529.3345,35103.08,46650.3845,0.455991,0.526948,zKiH8jXK5S7cw5j5N,CO,0.455991,0.578977,0.518397,0.541009,0.115825,105191.95,CO,63879.09,481.0,1.0,1.0,6.586941
4055,2022-01,71006.35,35784.23,81657.3025,35222.12,45873.0725,0.496042,0.561776,zKiH8jXK5S7cw5j5N,CO,0.496042,0.572597,0.497991,0.533655,-0.077624,109265.23,CO,63879.09,481.0,1.0,1.0,6.842003
4056,2022-02,74537.02,37413.09,85717.5730,37123.93,48304.4830,0.498060,0.563531,zKiH8jXK5S7cw5j5N,CO,0.498060,0.567273,0.483364,0.519047,0.049723,115076.27,CO,63879.09,481.0,1.0,1.0,7.205880
4057,2022-03,82826.62,37779.24,95250.6130,45047.38,57471.3730,0.543876,0.603370,zKiH8jXK5S7cw5j5N,CO,0.543876,0.565713,0.512659,0.515528,0.111215,110976.56,CO,63879.09,481.0,1.0,1.0,6.949163


# UW

In [44]:
uw_data = data[data['year_month'] == '2022-03']

In [45]:
uw_data_long = uw_data[uw_data['gm_6m'].notna()]

In [46]:
uw_data_long = uw_data_long.reset_index(drop = True)

## GM % score

In [47]:
def get_gm_perc_thresholds(state):
    if state == 'CA':
        return [[0.43,0.47,0.52,0.7],[0.43,0.47,0.52,0.7],[0.43,0.47,0.52,0.7]]
    elif state == 'CO':
        return [[0.36,0.45,0.53,0.7],[0.36,0.45,0.53,0.7],[0.36,0.45,0.53,0.7]]
    elif state == 'MI':
        return [[0.32,0.42,0.47,0.7],[0.32,0.42,0.47,0.7],[0.32,0.42,0.47,0.7]]
    elif state == 'MA':
        return [[0.45,0.48,0.49,0.7],[0.45,0.48,0.49,0.7],[0.45,0.48,0.49,0.7]]
    elif state == 'OR':
        return [[0.44,0.50,0.55,0.7],[0.44,0.50,0.55,0.7],[0.44,0.50,0.55,0.7]]
    else:
        return None
    

In [48]:
def get_gm_perc_scores(threshold,gm_3,gm_6,gm_all):
    # 3m score
    if gm_3 <= threshold[0][0]:
        score_3m = -2
    elif threshold[0][0] < gm_3 <= threshold[0][1]:
        score_3m = 0
    elif threshold[0][1] < gm_3 <= threshold[0][2]:
        score_3m = 5
    elif threshold[0][2] < gm_3 <= threshold[0][3]:
        score_3m = 5
    else:
        score_3m = 0
    # 6m score
    if gm_6 <= threshold[1][0]:
        score_6m = -2
    elif threshold[1][0] < gm_6 <= threshold[1][1]:
        score_6m = 0
    elif threshold[1][1] < gm_6 <= threshold[1][2]:
        score_6m = 5
    elif threshold[1][2] < gm_6 <= threshold[1][3]:
        score_6m = 5
    else:
        score_6m = 0
    # all time score
    if gm_all <= threshold[2][0]:
        score_all = -2
    elif threshold[2][0] < gm_all <= threshold[2][1]:
        score_all = 0
    elif threshold[2][1] < gm_all <= threshold[2][2]:
        score_all = 5
    elif threshold[2][2] < gm_all <= threshold[2][3]:
        score_all = 5
    else:
        score_all = 0
    return score_3m,score_6m,score_all
    

In [49]:
uw_data_long

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547


In [50]:
gm_score_data = uw_data_long.apply(lambda row: get_gm_perc_scores(get_gm_perc_thresholds(row['loc_state']),row['gm_3m'],row['gm_6m'],row['gm_9m']),axis = 1)
uw_data_long_gm = pd.concat([uw_data_long, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_3m_score', 'gm_6m_score','gm_all_score'])],axis=1)



In [51]:
uw_data_long_gm


,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124,-2,-2,-2
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144,-2,-2,-2
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416,5,5,5
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706,-2,-2,-2
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191,5,0,0
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061,5,5,0
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504,-2,-2,-2
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547,5,5,5


## inventory TO score

In [52]:
uw_data_long_gm['turnover_score'] = [10 if (uw_data_long_gm['inventory_to'][i] >= 6 and uw_data_long_gm['inventory_to'][i] <= 26) else 0 for i in range(len(uw_data_long_gm))]




In [53]:
uw_data_long_gm

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124,-2,-2,-2,0
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144,-2,-2,-2,0
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416,5,5,5,10
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706,-2,-2,-2,0
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387,5,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191,5,0,0,10
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061,5,5,0,0
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504,-2,-2,-2,0
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547,5,5,5,10


## rev change vs state score

In [54]:
data_with_rev_change = pd.merge(data,rev_change_bm_long_with_weight, on = ['year_month','location_state'])

In [55]:
data_with_rev_change['variance'] = data_with_rev_change['rev_change'] - data_with_rev_change['50%']

In [56]:
data_with_rev_change['rev_change_points'] = [gmv_change_variance_point_mapping(n) for n in data_with_rev_change['variance']]
data_with_rev_change['rev_change_total'] = data_with_rev_change['rev_change_points'] * data_with_rev_change['weight']



In [57]:
data_with_rev_change

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,50%,weight,variance,rev_change_points,rev_change_total
0,2021-05,198.00,99.00,227.7000,99.00,128.7000,0.500000,0.565217,22rkBxnqoaobkG9hr,CA,0.565217,0.565217,NaN,NaN,NaN,NaN,CA,31891.6700,272.0,1.000000,1.000000,NaN,-0.046000,0.20,NaN,NaN,NaN
1,2021-05,179951.33,78190.51,206944.0295,101760.82,128753.5195,0.565491,0.622166,2njkqeX7SW5XtzPXF,CA,0.622166,0.621084,0.616308,NaN,-0.051733,247488.13,CA,71378.9100,1034.0,1.000000,1.000000,13.868978,-0.046000,0.20,-0.005733,0.0,0.0
2,2021-05,166707.71,111350.58,191713.8665,55357.13,80363.2865,0.332061,0.419183,46c54b22qXp5R7sLm,CA,0.419183,-0.083702,-0.210860,NaN,-0.022363,737812.85,CA,457515.7850,2319.0,0.991873,1.000000,6.450600,-0.046000,0.20,0.023638,5.0,1.0
3,2021-05,88694.72,49360.50,101998.9280,39334.22,52638.4280,0.443479,0.516068,9qbx82jZwxnvdDDNX,CA,0.516068,0.515133,0.516311,NaN,-0.077289,157560.15,CA,93680.8900,632.0,1.000000,1.000000,6.727526,-0.046000,0.20,-0.031288,0.0,0.0
4,2021-05,692782.89,296880.84,796700.3235,395902.05,499819.4835,0.571466,0.627362,BjHoG26XcPdhAdraK,CA,0.627362,0.623432,0.618016,NaN,-0.176294,1117587.97,CA,358421.7950,1802.0,1.000000,1.000000,12.472322,-0.046000,0.20,-0.130293,-5.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351,2021-04,279165.98,132049.08,321040.8770,147116.90,188991.7970,0.526987,0.588685,NdrXf7rMG27rCT4XP,MA,0.526987,0.680829,0.653329,NaN,-0.522987,568578.51,MA,157565.4200,701.0,1.000000,0.915835,14.434094,0.018804,0.12,-0.541791,-5.0,-0.6
3352,2021-04,69152.84,39056.82,79525.7660,30096.02,40468.9460,0.435210,0.508878,R5EZkhoLBF9e7qxmF,MA,0.435210,0.437401,NaN,NaN,1.820990,NaN,MA,73663.4675,174.0,1.000000,1.000000,NaN,0.018804,0.12,1.802185,NaN,NaN
3353,2021-04,152.00,73.50,174.8000,78.50,101.3000,0.516447,0.579519,TgLFQQcK4NiY8unCE,MA,0.516447,0.516447,NaN,NaN,NaN,NaN,MA,497763.0900,753.0,1.000000,1.000000,NaN,0.018804,0.12,NaN,NaN,NaN
3354,2021-04,593098.29,382037.03,682063.0335,211061.26,300026.0035,0.355862,0.439880,dTY2B8G3GjHJEtLB6,MA,0.355862,0.440592,0.431717,NaN,0.006512,946779.30,MA,262435.7200,649.0,1.000000,1.000000,14.430647,0.018804,0.12,-0.012292,0.0,0.0


In [58]:
rev_change_score = data_with_rev_change[['location_id','rev_change_total']].groupby('location_id').sum().reset_index()

In [59]:
rev_change_score['rev_change_score'] = [max(min(round(rev_change_score['rev_change_total'][i],2),10),-10) for i in range(len(rev_change_score))]
rev_change_score.sort_values(by = 'rev_change_total')


,location_id,rev_change_total,rev_change_score
101,NNbgtFSgK2HDuoMyF,-14.1,-10.0
61,G4yyguo4R7wGp3NR8,-13.0,-10.0
70,Hcug7wxTBbgdcwuBp,-11.8,-10.0
212,jmbZL9JBgJpPYdRjM,-10.7,-10.0
9,43HM2Zu77qrMksWB8,-10.6,-10.0
...,...,...,...
286,wGeAKkDSeYQb22QhS,42.3,10.0
234,ofeoiXgSPuM6PCyaN,45.9,10.0
32,99hg27BQgm63LcWRn,46.6,10.0
10,46c54b22qXp5R7sLm,47.0,10.0


In [60]:
rev_change_score['rev_change_score'].describe()

count    305.000000
mean       6.896066
std        5.065107
min      -10.000000
25%        3.900000
50%       10.000000
75%       10.000000
max       10.000000
Name: rev_change_score, dtype: float64

In [61]:
uw_data_long_gm_rev = pd.merge(uw_data_long_gm,rev_change_score[['location_id','rev_change_score']], on = ['location_id'])

In [62]:
uw_data_long_gm_rev['rev_change_score'].describe()

count    247.000000
mean       8.088664
std        4.130133
min      -10.000000
25%        9.350000
50%       10.000000
75%       10.000000
max       10.000000
Name: rev_change_score, dtype: float64

In [63]:
uw_data_long_gm_rev


,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124,-2,-2,-2,0,10.0
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144,-2,-2,-2,0,10.0
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416,5,5,5,10,10.0
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706,-2,-2,-2,0,10.0
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387,5,0,0,10,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191,5,0,0,10,10.0
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061,5,5,0,0,10.0
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504,-2,-2,-2,0,10.0
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547,5,5,5,10,10.0


## gm $ score

In [64]:
gm_dollar_score = data[['location_id','subtotalInDollars_after_tax']].groupby('location_id').mean().reset_index()


In [65]:
gm_dollar_score['gm_dollar_score'] = [15 if gm_dollar_score['subtotalInDollars_after_tax'][i] >= 200000 else 0 for i in range(len(gm_dollar_score))]



In [66]:
gm_dollar_score['gm_dollar_score'].describe()

count    308.000000
mean       5.649351
std        7.279915
min        0.000000
25%        0.000000
50%        0.000000
75%       15.000000
max       15.000000
Name: gm_dollar_score, dtype: float64

In [67]:
final = pd.merge(uw_data_long_gm_rev,gm_dollar_score[['location_id','gm_dollar_score']],on = ['location_id'])

In [68]:
final

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124,-2,-2,-2,0,10.0,0
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144,-2,-2,-2,0,10.0,0
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416,5,5,5,10,10.0,0
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706,-2,-2,-2,0,10.0,15
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387,5,0,0,10,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191,5,0,0,10,10.0,15
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061,5,5,0,0,10.0,0
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504,-2,-2,-2,0,10.0,0
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547,5,5,5,10,10.0,0


# final 

In [69]:
final['total_gm_perc_score'] = final['gm_3m_score'] + final['gm_6m_score'] + final['gm_all_score']
final['total_gm_score'] = [min(final['gm_dollar_score'][i]+final['total_gm_perc_score'][i],15) for i in range(len(final))]
final['total_score'] = final['total_gm_score'] + final['rev_change_score'] + final['turnover_score']
final['credit_limit'] = [round(min(final['sum_3m_cogs'][i],final['inventory'][i]),0) for i in range(len(final))]
final['credit_limit_final'] = [min(max(final['credit_limit'][i],50000),250000) for i in range(len(final))]               



In [70]:
final

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit,credit_limit_final
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124,-2,-2,-2,0,10.0,0,-6,-6,4.0,21464.0,50000.0
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144,-2,-2,-2,0,10.0,0,-6,-6,4.0,304885.0,250000.0
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416,5,5,5,10,10.0,0,15,15,35.0,52438.0,52438.0
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706,-2,-2,-2,0,10.0,15,-6,9,19.0,939493.0,250000.0
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387,5,0,0,10,10.0,0,5,5,25.0,57761.0,57761.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191,5,0,0,10,10.0,15,5,15,35.0,126650.0,126650.0
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061,5,5,0,0,10.0,0,10,10,20.0,35936.0,50000.0
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504,-2,-2,-2,0,10.0,0,-6,-6,4.0,102185.0,102185.0
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547,5,5,5,10,10.0,0,15,15,35.0,81820.0,81820.0


In [71]:
final['total_score'].describe()

count    247.000000
mean      23.177733
std       11.503235
min       -7.900000
25%       14.450000
50%       25.000000
75%       35.000000
max       35.000000
Name: total_score, dtype: float64

In [72]:
final['pass'] = final['total_score'] >= 24

In [73]:
final

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit,credit_limit_final,pass
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124,-2,-2,-2,0,10.0,0,-6,-6,4.0,21464.0,50000.0,False
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144,-2,-2,-2,0,10.0,0,-6,-6,4.0,304885.0,250000.0,False
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416,5,5,5,10,10.0,0,15,15,35.0,52438.0,52438.0,True
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706,-2,-2,-2,0,10.0,15,-6,9,19.0,939493.0,250000.0,False
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387,5,0,0,10,10.0,0,5,5,25.0,57761.0,57761.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191,5,0,0,10,10.0,15,5,15,35.0,126650.0,126650.0,True
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061,5,5,0,0,10.0,0,10,10,20.0,35936.0,50000.0,False
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504,-2,-2,-2,0,10.0,0,-6,-6,4.0,102185.0,102185.0,False
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547,5,5,5,10,10.0,0,15,15,35.0,81820.0,81820.0,True


In [75]:
final['final_pass'] = (final['pass'] == True)&(final['profit_after_tax'] >= 25000) & (final['credit_limit'] >= 30000)


In [76]:
final

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit,credit_limit_final,pass,final_pass
0,2022-03,14291.48,9848.66,16435.2020,4442.82,6586.5420,0.310872,0.400758,22rkBxnqoaobkG9hr,CA,0.400758,0.367543,0.420930,0.313897,0.291735,21464.08,CA,31891.67000,272.0,1.000000,1.000000,2.692124,-2,-2,-2,0,10.0,0,-6,-6,4.0,21464.0,50000.0,False,False
1,2022-03,181592.73,103940.35,208831.6395,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.50000,528.0,1.000000,1.000000,3.801144,-2,-2,-2,0,10.0,0,-6,-6,4.0,304885.0,250000.0,False,False
2,2022-03,110459.05,39999.50,127027.9075,70459.55,87028.4075,0.637879,0.685113,2yBx7nstyQ8ZsAmJr,CO,0.637879,0.642193,0.628788,0.634540,0.183986,113664.51,CO,52437.86000,457.0,1.000000,0.910284,8.670416,5,5,5,10,10.0,0,15,15,35.0,52438.0,52438.0,True,True
3,2022-03,669152.23,420858.87,769525.0645,248293.36,348666.1945,0.371057,0.453093,35pY3nxKPGdfQKT28,CO,0.371057,0.306344,0.295962,0.281009,0.083673,1347399.32,CO,939493.34970,1162.0,1.000000,1.000000,5.736706,-2,-2,-2,0,10.0,15,-6,9,19.0,939493.0,250000.0,False,False
4,2022-03,148254.89,71873.73,170493.1235,76381.16,98619.3935,0.515202,0.578436,3KTQcT9Stpofdf5kB,OR,0.515202,0.493774,0.509127,0.492854,0.046938,211325.77,OR,57761.42600,753.0,1.000000,1.000000,14.634387,5,0,0,10,10.0,0,5,5,25.0,57761.0,57761.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2022-03,191379.13,95071.00,220085.9995,96308.13,125014.9995,0.503232,0.568028,yji6wnh5kT8GZmM26,OR,0.503232,0.495098,0.504935,0.488473,0.091813,266160.28,OR,126649.63988,1087.0,0.997248,0.999080,8.406191,5,0,0,10,10.0,15,5,15,35.0,126650.0,126650.0,True,True
243,2022-03,35702.64,16089.79,41058.0360,19612.85,24968.2460,0.549339,0.608121,yrR9pHpbacgffGFpZ,OR,0.549339,0.477398,0.551407,0.524547,-0.070090,51693.85,OR,35935.55640,540.0,1.000000,1.000000,5.754061,5,5,0,0,10.0,0,10,10,20.0,35936.0,50000.0,False,False
244,2022-03,56081.28,33506.59,64493.4720,22574.69,30986.8820,0.402535,0.480465,yxhFg37pDhFC5MTvb,OR,0.402535,0.389646,0.401578,0.406208,0.135739,102185.21,OR,111540.56656,654.0,1.000000,1.000000,3.664504,-2,-2,-2,0,10.0,0,-6,-6,4.0,102185.0,102185.0,False,False
245,2022-03,147722.01,63917.76,169880.3115,83804.25,105962.5515,0.567311,0.623748,zHcRaoDBLFosu3PdC,OR,0.567311,0.554749,0.569902,0.561499,0.017626,188503.90,OR,81819.95000,976.0,1.000000,1.000000,9.215547,5,5,5,10,10.0,0,15,15,35.0,81820.0,81820.0,True,True


In [80]:
final.groupby(['loc_state'])['final_pass'].mean()

loc_state
CA    0.645161
CO    0.518987
MA    0.181818
MI    0.510638
OR    0.632911
Name: final_pass, dtype: float64

In [99]:
final[final['loc_state'] == 'MA']

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit,credit_limit_final,pass,final_pass
1,2022-03,181592.73,103940.35,2.088316e+05,77652.38,104891.2895,0.427618,0.502277,2t3guxpfanCWqycA6,MA,0.427618,0.412441,0.422982,0.416785,0.106398,304885.49,MA,320835.5000,528.0,1.0,1.000000,3.801144,-2,-2,-2,0,10.0,0,-6,-6,4.0,304885.0,250000.0,False,False
9,2022-03,1105115.58,592890.58,1.270883e+06,512225.00,677992.3370,0.463504,0.533481,4hmkTc8KYoncMnH59,MA,0.463504,0.450655,0.351293,0.410773,-0.053332,2256949.51,MA,422986.6900,641.0,1.0,1.000000,21.342984,-2,-2,0,10,-2.4,15,-4,11,18.6,422987.0,250000.0,False,False
13,2022-03,570910.74,291324.54,6.565474e+05,279586.20,365222.8110,0.489720,0.556278,5c49T5Mh8FNuWpBG2,MA,0.489720,0.493812,0.499989,0.499357,0.091579,834789.93,MA,547942.2300,584.0,1.0,1.000000,6.094000,5,5,5,10,3.3,15,15,15,28.3,547942.0,250000.0,True,True
79,2022-03,142653.26,86004.30,1.640512e+05,56648.96,78046.9490,0.397109,0.475747,NdrXf7rMG27rCT4XP,MA,0.397109,0.499707,0.392631,0.393221,0.106813,231589.47,MA,157565.4200,701.0,1.0,0.915835,5.879195,-2,-2,5,0,2.7,15,1,15,17.7,157565.0,157565.0,False,False
92,2022-03,72018.74,37062.17,8.282155e+04,34956.57,45759.3810,0.485382,0.552506,R5EZkhoLBF9e7qxmF,MA,0.485382,0.465604,0.472561,0.479980,0.033819,112802.19,MA,73663.4675,174.0,1.0,1.000000,6.125272,0,0,0,10,7.1,0,0,0,17.1,73663.0,73663.0,False,False
103,2022-03,423610.63,206920.42,4.871522e+05,216690.21,280231.8045,0.511532,0.575245,TgLFQQcK4NiY8unCE,MA,0.511532,0.507213,0.515324,0.514092,0.053419,603117.31,MA,497763.0900,753.0,1.0,1.000000,4.846621,5,5,5,0,-1.2,15,15,15,13.8,497763.0,250000.0,False,False
138,2022-03,56194.33,32931.67,6.462348e+04,23262.66,31691.8095,0.413968,0.490407,cTH4YN7E3zYmJdL5B,MA,0.413968,0.392641,0.414186,0.392641,0.534072,67671.40,MA,56015.8300,310.0,1.0,0.961290,4.832305,-2,-2,-2,0,10.0,0,-6,-6,4.0,56016.0,56016.0,False,False
143,2022-03,522670.26,263159.09,6.010708e+05,259511.17,337911.7090,0.496510,0.562183,dTY2B8G3GjHJEtLB6,MA,0.496510,0.476259,0.501076,0.499809,0.071373,769071.53,MA,262435.7200,649.0,1.0,1.000000,11.722056,5,5,0,10,-1.2,15,10,15,23.8,262436.0,250000.0,False,False
165,2022-03,131369.51,72608.80,1.510749e+05,58760.71,78466.1365,0.447293,0.519386,hoL6QWpKg4mFEEiB2,MA,0.447293,0.432882,0.464435,0.471820,-0.003819,202128.99,MA,251435.5250,520.0,1.0,1.000000,3.215600,0,0,-2,0,10.0,0,-2,-2,8.0,202129.0,202129.0,False,False
211,2022-03,238260.06,123740.01,2.739991e+05,114520.05,150259.0590,0.480651,0.548393,sppgLxeycvxreFGan,MA,0.480651,0.482766,0.475396,0.463226,0.118456,347516.30,MA,321320.6900,352.0,1.0,1.000000,4.326099,0,0,5,0,10.0,0,5,5,15.0,321321.0,250000.0,False,False


In [100]:
data_with_rev_change[data_with_rev_change['location_id'] == '5c49T5Mh8FNuWpBG2' ]

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,50%,weight,variance,rev_change_points,rev_change_total
521,2021-05,670758.74,345260.05,771372.5510,325498.69,426112.5010,0.485269,0.552408,5c49T5Mh8FNuWpBG2,MA,0.485269,0.493892,0.486397,NaN,-0.000218,1026624.15,MA,547942.23,584.0,1.0,1.0,7.494397,0.065254,0.20,-0.065472,-2.5,-0.5
529,2021-06,602442.60,306929.03,692808.9900,295513.57,385879.9600,0.490526,0.556979,5c49T5Mh8FNuWpBG2,MA,0.490526,0.493331,0.484911,0.493331,-0.101849,1001772.61,MA,547942.23,584.0,1.0,1.0,7.312980,-0.054168,0.28,-0.047681,0.0,0.0
537,2021-07,672305.21,358199.74,773150.9915,314105.47,414951.2515,0.467207,0.536701,5c49T5Mh8FNuWpBG2,MA,0.467207,0.489599,0.481001,0.486149,0.115966,1010388.82,MA,547942.23,584.0,1.0,1.0,7.375878,0.231451,0.36,-0.115486,-5.0,-1.8
545,2021-08,657754.57,339464.78,756417.7555,318289.79,416952.9755,0.483904,0.551221,5c49T5Mh8FNuWpBG2,MA,0.483904,0.488887,0.480545,0.483471,-0.021643,1004593.55,MA,547942.23,584.0,1.0,1.0,7.333573,0.015651,0.44,-0.037294,0.0,0.0
553,2021-09,599121.72,299596.13,688989.9780,299525.59,389393.8480,0.499941,0.565166,5c49T5Mh8FNuWpBG2,MA,0.499941,0.490115,0.483684,0.484297,-0.089141,997260.65,MA,547942.23,584.0,1.0,1.0,7.280042,-0.086423,0.52,-0.002717,0.0,0.0
564,2021-10,607985.88,305216.22,699183.7620,302769.66,393967.5420,0.497988,0.563468,5c49T5Mh8FNuWpBG2,MA,0.497988,0.490902,0.493944,0.487472,0.014795,944277.13,MA,547942.23,584.0,1.0,1.0,6.893260,0.051905,0.60,-0.037110,0.0,0.0
574,2021-11,558519.85,287379.25,642297.8275,271140.60,354918.5775,0.485463,0.552576,5c49T5Mh8FNuWpBG2,MA,0.485463,0.490408,0.494464,0.487505,-0.081360,892191.60,MA,547942.23,584.0,1.0,1.0,6.513034,-0.019048,0.68,-0.062313,-2.5,-1.7
585,2021-12,638166.50,310961.11,733891.4750,327205.39,422930.3650,0.512727,0.576285,5c49T5Mh8FNuWpBG2,MA,0.512727,0.492268,0.498726,0.491205,0.142603,903556.58,MA,547942.23,584.0,1.0,1.0,6.595999,0.121731,0.76,0.020872,5.0,3.8
597,2022-01,575848.02,281315.75,662225.2230,294532.27,380909.4730,0.511476,0.575196,5c49T5Mh8FNuWpBG2,MA,0.511476,0.493745,0.503222,0.498583,-0.097652,879656.11,MA,547942.23,584.0,1.0,1.0,6.421524,-0.094010,0.84,-0.003642,0.0,0.0
610,2022-02,523013.89,262149.64,601465.9735,260864.25,339316.3335,0.498771,0.564149,5c49T5Mh8FNuWpBG2,MA,0.498771,0.494104,0.507658,0.501061,-0.091750,854426.50,MA,547942.23,584.0,1.0,1.0,6.237347,0.007610,0.92,-0.099361,-2.5,-2.3


In [84]:
final[(final['loc_state'] == 'MA')&(final['pass'] == True)]

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs,loc_state,inventory,num_sku_in_inventory,coverage,non_zero_cost_perc,inventory_to,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit,credit_limit_final,pass,final_pass
13,2022-03,570910.74,291324.54,656547.3510,279586.20,365222.8110,0.489720,0.556278,5c49T5Mh8FNuWpBG2,MA,0.489720,0.493812,0.499989,0.499357,0.091579,834789.93,MA,547942.23,584.0,1.0,1.0,6.094000,5,5,5,10,3.3,15,15,15,28.3,547942.0,250000.0,True,True
236,2022-03,749966.93,347829.06,862461.9695,402137.87,514632.9095,0.536207,0.596702,wrqGh9NmHyqjwnJQv,MA,0.536207,0.496422,0.531406,0.519015,0.089600,988407.63,MA,304896.52,461.0,1.0,1.0,12.967122,5,5,5,10,10.0,15,15,15,35.0,304897.0,250000.0,True,True


In [86]:
fh_data[fh_data['location_id'] == '2t3guxpfanCWqycA6']

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs
26,2021-05,289.36,197.23,332.7640,92.13,135.5340,0.318392,0.407298,2t3guxpfanCWqycA6,MA,0.318392,0.318392,NaN,NaN,NaN,NaN
27,2021-06,62552.52,36680.77,71935.3980,25871.75,35254.6280,0.413600,0.490087,2t3guxpfanCWqycA6,MA,0.413600,0.365996,NaN,NaN,215.175422,NaN
28,2021-07,88707.09,50548.01,102013.1535,38159.08,51465.1435,0.430169,0.504495,2t3guxpfanCWqycA6,MA,0.430169,0.387387,0.387387,NaN,0.418122,87426.01
29,2021-08,101400.09,57230.99,116610.1035,44169.10,59379.1135,0.435592,0.509211,2t3guxpfanCWqycA6,MA,0.435592,0.399439,0.426454,NaN,0.143089,144459.77
30,2021-09,128840.31,72357.84,148166.3565,56482.47,75808.5165,0.438391,0.511645,2t3guxpfanCWqycA6,MA,0.438391,0.407229,0.434718,NaN,0.270613,180136.84
31,2021-10,160665.47,93426.81,184765.2905,67238.66,91338.4805,0.418501,0.494349,2t3guxpfanCWqycA6,MA,0.418501,0.409108,0.430828,0.409108,0.247012,223015.64
32,2021-11,167869.47,99780.66,193049.8905,68088.81,93269.2305,0.405606,0.483135,2t3guxpfanCWqycA6,MA,0.405606,0.408607,0.420833,0.423643,0.044839,265565.31
33,2021-12,193075.43,114367.40,222036.7445,78708.03,107669.3445,0.407654,0.484917,2t3guxpfanCWqycA6,MA,0.407654,0.408488,0.410587,0.422652,0.150152,307574.87
34,2022-01,182502.72,107008.20,209878.1280,75494.52,102869.9280,0.413662,0.490141,2t3guxpfanCWqycA6,MA,0.413662,0.409063,0.408974,0.419901,-0.054759,321156.26
35,2022-02,164129.65,93936.94,188749.0975,70192.71,94812.1575,0.427666,0.502318,2t3guxpfanCWqycA6,MA,0.427666,0.410924,0.416328,0.418580,-0.100673,315312.54


In [87]:
103940.35 + 93936.94 + 107008.20

304885.49

In [91]:
fh_data[fh_data['location_state'] == 'MA'].isna().sum()

year_month                      0
subtotalInDollars               0
costInDollars                   0
subtotalInDollars_after_tax     0
profit                          0
profit_after_tax                0
margin_perc                     0
margin_perc_after_tax           0
location_id                     0
location_state                  0
margin_final                    0
gm_9m                           0
gm_3m                          26
gm_6m                          63
rev_change                     13
sum_3m_cogs                    26
dtype: int64

In [92]:
fh_data[fh_data['location_state'] == 'MA']

,year_month,subtotalInDollars,costInDollars,subtotalInDollars_after_tax,profit,profit_after_tax,margin_perc,margin_perc_after_tax,location_id,location_state,margin_final,gm_9m,gm_3m,gm_6m,rev_change,sum_3m_cogs
26,2021-05,289.36,197.23,332.7640,92.13,135.5340,0.318392,0.407298,2t3guxpfanCWqycA6,MA,0.318392,0.318392,NaN,NaN,NaN,NaN
27,2021-06,62552.52,36680.77,71935.3980,25871.75,35254.6280,0.413600,0.490087,2t3guxpfanCWqycA6,MA,0.413600,0.365996,NaN,NaN,215.175422,NaN
28,2021-07,88707.09,50548.01,102013.1535,38159.08,51465.1435,0.430169,0.504495,2t3guxpfanCWqycA6,MA,0.430169,0.387387,0.387387,NaN,0.418122,87426.01
29,2021-08,101400.09,57230.99,116610.1035,44169.10,59379.1135,0.435592,0.509211,2t3guxpfanCWqycA6,MA,0.435592,0.399439,0.426454,NaN,0.143089,144459.77
30,2021-09,128840.31,72357.84,148166.3565,56482.47,75808.5165,0.438391,0.511645,2t3guxpfanCWqycA6,MA,0.438391,0.407229,0.434718,NaN,0.270613,180136.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3890,2021-12,758049.65,367387.75,871757.0975,390661.90,504369.3475,0.515351,0.578566,wrqGh9NmHyqjwnJQv,MA,0.515351,0.487676,0.506624,0.499812,-0.077904,1536497.85
3891,2022-01,671876.43,318719.91,772657.8945,353156.52,453937.9845,0.525627,0.587502,wrqGh9NmHyqjwnJQv,MA,0.525627,0.490596,0.514189,0.506022,-0.113678,1095848.32
3892,2022-02,688295.62,321858.66,791539.9630,366436.96,469681.3030,0.532383,0.593377,wrqGh9NmHyqjwnJQv,MA,0.532383,0.493580,0.524454,0.511885,0.024438,1007966.32
3893,2022-03,749966.93,347829.06,862461.9695,402137.87,514632.9095,0.536207,0.596702,wrqGh9NmHyqjwnJQv,MA,0.536207,0.496422,0.531406,0.519015,0.089600,988407.63


In [110]:
final.to_csv('flowhub_pre_approval_0427.csv')